In [0]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
#from keras import backend as K
import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, Dropout, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Conv1D
from keras.models import Model, Sequential, load_model
#from keras.applications.resnext import preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.applications.inception_v3 import InceptionV3
from google.colab import files

uploaded = files.upload()

"""!wget --no-check-certificate \
    https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/downloads/chest-xray-pneumonia.zip/2 \
    -O /tmp/chest-xray-pneumonia.zip
"""
#local_zip = open("chest-xray-pneumonia.zip").read()


#model = InceptionV3(weights = "imagenet", include_top = False, classes = 2)
  



Saving chest_xray.zip to chest_xray (1).zip


'!wget --no-check-certificate     https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia/downloads/chest-xray-pneumonia.zip/2     -O /tmp/chest-xray-pneumonia.zip\n'

In [0]:
from zipfile import ZipFile
import zipfile

local_zip = zipfile.ZipFile("chest_xray.zip")

zip_ref = ZipFile(local_zip)
zip_ref = zip_ref.extractall("chest_xray.zip")
zip_ref.close()

In [0]:
import os
#listing training and testing data.
Train_Dir = os.listdir("/tmp/chest_xray/train/") 
Test_Dir = os.listdir("/tmp/chest_xray/test/")
#in all images of training data.
print(os.listdir("/tmp/chest_xray/train/"))
print(os.listdir("/tmp/chest_xray/test/"))


#Making training and testing data labelling.
def labeling(dir, test):
  for file in os.listdir(dir):
    if not files.startswith("."):
      if files in ["NORMAL"]:
        label = 0
      elif files in ["PNEUMONIA"]:
        label = 1
      else:
        label = 2
        
  return file, label


In [0]:
def get_data(dir):
  x = []
  y = []
  
  for file in os.listdir(dir):
    if not file.startswith("."):
      if file in ["NORMAL"]:
        label = 0
      elif file in ["PNEUMONIA"]:
        label = 1
      else:
        label = 2
      temp = dir + file
      for file in tqdm(os.listdir(temp)):
        image = cv2.imread(temp + "/" + file)
        if image is not None:
          image = cv2.resize(image, (150, 150, 3))
          image = np.asarray(image)
          x.append(image)
          y.append(label)
  
  x = np.asarray(x)
  y = np.asarray(y)
  
  
  return x, y

In [0]:
#Data Pre-processing.
#Resizing and puttin in all labels so prepared from the previous function to the training and testing data
#into the respective labels.

import skimage
from skimage.transform import resize

def data_preprocessing(file, label):
  #Making an empty list for dataset and labels 
  x = []
  y = []
  
  for file in os.listdir(dir):
    file, label = labelling(dir)
    temp = dir + file
    
    for image_filename in tqdm(os.listdir(temp)):
      path = os.listdir(temp + "/" + image_filename)
      image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
      if image is not None:
        image = skimage.transform.resize(image, (150, 150, 3))
        image = np.asarray(image)
        x.append(image)
        y.append(label)
  

  #Convert the images and labels into arrays.
  x = np.asarray(x)
  y = np.asarray(y)

  
  
  return x, y 
 

In [0]:
x_train, y_train = labelling(Train_Dir)

In [0]:
x_test, y_test = labelling(Test_Dir)

In [0]:
x_train, y_train = data_preprocessing(Train_Dir)

In [0]:
x_test, y_test = data_preprocessing(Test_Dir)

In [0]:
x_train, y_train = get_data(Train_Dir)

In [0]:
x_test, y_test = get_data(Train_Dir)

In [0]:
print(x_train.shape + "\n" + x_test.shape)
print(y_train.shape + "\n" + y_test.shape)

In [0]:
from keras.utils.np_utils import to_categorical

y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [0]:
import seaborn as sns
x = y_train.sum(axis = 0)
sns.countplot(x = count)

In [0]:
x_train = x_train.reshape(5216, 3, 150, 150)
x_test = x_test.reshape(624, 3, 150, 150)

In [0]:
model = Sequential()
model.add(Conv2D(16, kernel_size = (3, 3), strides = (1, 1),activation = "relu", padding = "valid", input_shape = (3, 150, 150)))
model.add(Conv2D(16, kernel_size = (3, 3), padding = "same", activation = "relu"))
model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = "relu", padding = "same"))
model.add(Conv2D(32, (3, 3), activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(96, (3, 3), activation = "relu", padding = "same"))
model.add(Conv2D(96, (3, 3), activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), dilation_rate = (2, 2), activation = "relu", padding = "same"))
model.add(Conv2D(128, (3, 3), dilation_rate = (2, 2), activation = "relu", padding = "same"))
model.add(Conv2D(128, (3, 3), dilation_rate = (2, 2), activation = "relu", padding = "same"))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(BatchNormalization())

model.add(Flatten())

model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(2, actiation = "sigmoid"))


In [0]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
model.summary()

In [0]:
history = model.fit(x_train, y_train, validation_data = (x_test, y_test), batch_size = 32,epochs = 3)

In [0]:
plt.plot(history.history["acc"])
plt.plot(history.history["val_acc"])
plt.xlabel("loss")
plt.ylabel("epoch")
plt.title("model accuracy")
plt.legend(["train", "test"], loc = "upper left")
plt.show()

plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.xlabel("loss")
plt.ylabel("epoch")
plt.title("model loss")
plt.legend(["train", "test"], loc = "upper left")
plt.show()